In [3]:
import os
import cv2
import numpy as np
from datetime import timedelta
from matplotlib import pyplot as plt
import matplotlib

In [4]:
data_loc = "compressor-data/"
files = os.listdir(data_loc)

files_len = len(files)
print(files_len)

MAX_FRAMES_PER_SECOND = 10

803


In [5]:
def format_timedelta(td):
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":","-")

In [6]:
def get_saving_frames_durations(cap, saving_fps):
    s = []
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)

    for i in np.arange(0, clip_duration, 1/saving_fps):
        s.append(i)
    
    return s

In [24]:
def video_analyze(file, override=False):
    video = cv2.VideoCapture(file)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Total frames = " + str(total_frames))

    fps = video.get(cv2.CAP_PROP_FPS)
    print("FPS = " + str(fps))

In [25]:
file_name = data_loc + files[0]
video_analyze(file_name)

Total frames = 380
FPS = 10.0


In [26]:
def video_into_frames(video_file):
    filename, _ = os.path.splitext(video_file)
    filename += "-opencv"

    if not os.path.isdir(filename):
        os.mkdir(filename)
    
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    saving_frames_per_second = min(fps, MAX_FRAMES_PER_SECOND)
    saving_frames_duration = get_saving_frames_durations(cap, saving_frames_per_second)

    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            break
            
        frame_duration = count/fps
        try:
            closest_duration = saving_frames_duration[0]
        except IndexError:
            break

        if frame_duration >= closest_duration:
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(os.path.join(filename, f"frame{frame_duration_formatted}.jpg"), frame)

            try:
                saving_frames_duration.pop(0)
            except IndexError:
                pass
        
        count += 1

In [27]:
video_into_frames(data_loc + "20210628-081000.03.mp4")

In [7]:
from IPython.display import Image

In [29]:
# contours, hier = cv2.findContours(im_gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
# result = cv2.imread("gray.png")

# bubbles = []
# for cnt in contours:
#     if 0<cv2.contourArea(cnt)<300:
#         cv2.drawContours(original_image, cnt, -1, (0,0,255), 2)
#         bubbles.append(cnt)


# cv2.imwrite("result.png", original_image)
# Image(filename="result.png")


In [69]:
def image_crop(image):
    black_mask = np.zeros_like(image)
    black_mask[:,:] = [255,255,255]

    # Cover top
    start_point = (0,0)
    end_point = (1280,90)
    color = (0,0,0)

    black_mask = cv2.rectangle(black_mask, start_point, end_point, color, -1)
    image = cv2.bitwise_and(image, black_mask)

    start_point = (0, 450)
    end_point = (1280, 720)
    black_mask = cv2.rectangle(black_mask, start_point, end_point, color, -1)
    image = cv2.bitwise_and(image, black_mask)

    return image

In [151]:
def grayscale_blur(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(image, (7,7), 0)
    th, image = cv2.threshold(image, 127, 255, 0)

    return image

In [254]:
from collections import OrderedDict
class CentroidsList():
    def __init__(self):
        self.objectID = 0
        self.objects = OrderedDict()
        self.distance = OrderedDict()

    def load_centroids(self, centroids):
        for i in range (0,len(centroids)):
            self.objects[self.objectID] = centroids[i]
            self.distance[self.objectID] = 0
            self.objectID += 1

    def update_centroids(self, centroids):
        if len(self.objects) == 0:
            self.load_centroids(centroids)
        else:
            for centroid in centroids:
                min_dist = 0
                idx = 0
                # Calculate and update distance moved from previous centroid
                for id, old_centroid in self.objects.items():
                    dist = np.linalg.norm(centroid - old_centroid)
                    if min_dist == 0:
                        min_dist = dist
                        idx = id

                    if dist < min_dist:
                        min_dist = dist
                        self.distance[id] = min_dist

                        # Save ID of most minimum distance centroid
                        idx = id

            #  Update new centroid

                self.objects[idx] = centroid
                

        return self.objects, self.distance

In [255]:
def blob_extraction(image, result, c):
    result = cv2.connectedComponentsWithStats(result, 4, cv2.CV_32S)
    (numLabels, labels, stats, centroids) = result
    objects, distance = c.update_centroids(centroids)

    for (objectID_0, centroid) in objects.items():
        # text = "ID {}".format(objectID_0)
        # cv2.putText(image, text, (int(centroid[0]-10), int(centroid[1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        cv2.circle(image, (int(centroid[0]), int(centroid[1])), 5, (0, 0, 255), 1)

        for (objectID, object_distance) in distance.items():
            if objectID_0 == objectID:
                distance_text = "{}".format(object_distance)
                cv2.putText(image, distance_text, (int(centroid[0]-10), int(centroid[1]+30)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
    
    return image

In [256]:
folder_path = "compressor-data/" + "20210628-081000.03-opencv/"
output_path = "compressor-data/" + "20210628-081000.03-opencv-output/"
image_list = sorted(os.listdir(folder_path))
c = CentroidsList()
for file in image_list:
    if str(file)[-3:] == "jpg":
        image = cv2.imread(folder_path + file)
        result = image_crop(image)
        result = grayscale_blur(result)
        image = blob_extraction(image, result, c)
        cv2.imwrite(os.path.join(output_path, file), image)

In [153]:
image_array = []
for file in sorted(os.listdir(output_path)):
    if str(file)[-3:] == "jpg":
        image = cv2.imread(output_path + file)
        height, width, layers = image.shape
        size = (width, height)
        image_array.append(image)

out = cv2.VideoWriter(output_path + 'result.avi', cv2.VideoWriter_fourcc(*'DIVX'), 10, size)

for i in range(len(image_array)):
    out.write(image_array[i])

out.release()